In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import os
from orix.io import load
from orix.vector import Vector3d
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["axes.unicode_minus"] = False

In [ ]:

folder = "/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/SPED"
file = "SPED_640x383x12_20x20_9p26x9p26nm_ScanRot20deg_CL12cm_spot0p5nm_alpha5_NBD_"\
        "10ms_1deg_fixed_scan.zspy"


dp = hs.load(os.path.join(folder, file), lazy = True)

xmap_file = "/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/"\
        "Processed/One_deg/error_fixed/ErMnO3_1_deg_scan_errors_fixed_rotated.hdf5"

In [ ]:
def load_and_preprocess(file, stepsize = 6.92):
    xmap = load(file)
    y, x = np.indices(xmap.shape, dtype="float")
    x *= stepsize
    y *= stepsize
    xmap._x = x.ravel()
    xmap._y = y.ravel()
    xmap.scan_unit = "nm"

    return xmap

xmap = load_and_preprocess(xmap_file)

In [ ]:
#express c axis in sample coordinate

mask = (xmap.correlation[:,0] > 0.001).reshape(383,640,1)

vecs = ~xmap.orientations*Vector3d.zvector()
vecs = vecs.data.reshape(xmap.shape + (3,)) *  mask

In [ ]:
#calculate angle between c-axis and plane

angles = np.arctan(vecs[:,:,2]/(np.sqrt(vecs[:,:,0]**2 + vecs[:,:,1]**2)))

#set angle for regions outside grains to pi/2

angles[mask.squeeze() == False] = np.pi/2


In [ ]:
#plot with hyperspy

img = hs.signals.Signal2D(np.rad2deg(angles))

img.axes_manager[0].offset = dp.axes_manager[0].offset
img.axes_manager[1].offset = dp.axes_manager[1].offset
img.axes_manager[0].scale = 6.92
img.axes_manager[1].scale = 6.92
img.axes_manager[0].units = dp.axes_manager[0].units
img.axes_manager[1].units = dp.axes_manager[1].units
img.axes_manager[0].name = dp.axes_manager[0].name
img.axes_manager[1].name = dp.axes_manager[1].name

img.plot(title = "", cmap ="viridis_r", axes_off = True)
#plt.savefig("c_axis_angle.svg")